## Qwen 2.5 Function Calling 예제

In [1]:
import json
import re
import yfinance as yf
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from datetime import datetime

INFO 04-21 08:55:17 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# 모델 및 토크나이저 초기화
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"
model = LLM(MODEL_ID, tensor_parallel_size=1)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

INFO 04-21 08:55:41 [config.py:600] This model supports multiple tasks: {'score', 'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 04-21 08:55:41 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-21 08:55:44 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_e

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-21 08:57:07 [loader.py:447] Loading weights took 77.96 seconds
INFO 04-21 08:57:08 [gpu_model_runner.py:1273] Model loading took 14.2487 GiB and 79.402409 seconds
INFO 04-21 08:57:24 [backends.py:416] Using cache directory: /giant-data/user/1111332/.cache/vllm/torch_compile_cache/450ea10ccb/rank_0_0 for vLLM's torch.compile
INFO 04-21 08:57:24 [backends.py:426] Dynamo bytecode transform time: 16.04 s
INFO 04-21 08:57:25 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-21 08:57:38 [monitor.py:33] torch.compile takes 16.04 s in total
INFO 04-21 08:57:39 [kv_cache_utils.py:578] GPU KV cache size: 951,888 tokens
INFO 04-21 08:57:39 [kv_cache_utils.py:581] Maximum concurrency for 32,768 tokens per request: 29.05x
INFO 04-21 08:58:14 [gpu_model_runner.py:1608] Graph capturing finished in 35 secs, took 1.45 GiB
INFO 04-21 08:58:14 [core.py:162] init engine (profile, create kv cache, warmup model) took 66.15 seconds


In [3]:
# 샘플링 파라미터 설정 - Qwen 2.5에 맞게 stop token 변경
sampling_params_func_call = SamplingParams(
    max_tokens=256, temperature=0.0, stop=["<|im_end|>"], skip_special_tokens=False
)
sampling_params_text = SamplingParams(
    max_tokens=512, temperature=0.1, top_p=0.95, stop=["<|im_end|>"], skip_special_tokens=False
)

In [4]:
# KOSPI 주식 정보
KOSPI_TICKER_MAP = {
    "SK텔레콤": "017670.KS", "삼성전자": "005930.KS", "SK하이닉스": "000660.KS",
    "현대차": "005380.KS", "기아": "000270.KS", "LG에너지솔루션": "373220.KS",
    "NAVER": "035420.KS", "카카오": "035720.KS",
}

In [5]:
# 도구(함수) 정의
TOOLS = [{
    "type": "function",
    "function": {
        "name": "get_kospi_stock_info",
        "description": "특정 KOSPI 주식의 현재 가격 및 기본 정보를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_name_or_code": {
                    "type": "string",
                    "description": "주식 이름(예: 'SK텔레콤') 또는 종목 코드(예: '017670')"
                }
            },
            "required": ["stock_name_or_code"]
        }
    }
}]

In [6]:
# 주가 조회
def get_kospi_stock_info(stock_name_or_code: str) -> str:

    name_or_code = stock_name_or_code.strip()
    ticker_symbol = None

    if re.fullmatch(r'\d{6}', name_or_code):
        ticker_symbol = name_or_code + ".KS"
    elif name_or_code in KOSPI_TICKER_MAP:
        ticker_symbol = KOSPI_TICKER_MAP[name_or_code]
    else:
        ticker_symbol = name_or_code + ".KS"

    stock = yf.Ticker(ticker_symbol)
    stock_info = stock.info

    current_price = stock_info.get('currentPrice')
    previous_close = stock_info.get('previousClose')

    price_to_use = current_price if current_price is not None else previous_close
    price_display = round(price_to_use, 2) if price_to_use is not None else "정보 없음"
    previous_close_display = round(previous_close, 2) if previous_close is not None else "정보 없음"

    result = {
        "ticker": ticker_symbol,
        "stock_name": stock_info.get('shortName', name_or_code),
        "current_price": price_display,
        "previous_close": previous_close_display,
        "currency": stock_info.get('currency', 'KRW')
    }
    
    return json.dumps(result, ensure_ascii=False)

In [7]:
# 함수 호출 파싱
# Qwen LLM 출력에서 <tool_call>...</tool_call> 형식의 함수 호출을 파싱
def parse_tool_calls(content: str):
    
    tool_calls = []
    pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.finditer(pattern, content, re.DOTALL)

    last_match_end = 0
    parsed_calls = []

    for match in matches:
        tool_call_content = match.group(1).strip()
        func_data = json.loads(tool_call_content)

        if isinstance(func_data.get("arguments"), str):
            func_data["arguments"] = json.loads(func_data.get("arguments"))

        parsed_calls.append({
            "type": "function",
            "function": {
                "name": func_data.get("name"),
                "arguments": func_data.get("arguments", {})
            },
            "id": f"call_{match.start()}"
        })
        last_match_end = match.end()

    first_match_start = content.find("<tool_call>")
    prefix_text = content[:first_match_start].strip() if first_match_start != -1 else content.strip()
    if not parsed_calls:
        prefix_text = re.sub(r"<\|im_end\|>\s*$", "", prefix_text).strip()

    assistant_message = {"role": "assistant"}
    if prefix_text:
        assistant_message["content"] = prefix_text
    if parsed_calls:
        assistant_message["tool_calls"] = parsed_calls
    if not prefix_text and not parsed_calls:
         assistant_message["content"] = ""

    if "content" in assistant_message and assistant_message["content"]:
        assistant_message["content"] = re.sub(r"<\|im_end\|>\s*$", "", assistant_message["content"]).strip()

    return assistant_message

In [8]:
# 메인 쿼리 처리 함수 
def query_kospi_info(query: str) -> str:
    current_date = datetime.now().strftime('%Y-%m-%d')
    messages = [
        {"role": "system", "content": f"You are a helpful assistant. Current Date: {current_date}"},
        {"role": "user", "content": query}
    ]
    print(f"\n### 1단계 초기 메시지 작성:\n{messages}")

    prompt = tokenizer.apply_chat_template(
        messages, tools=TOOLS, add_generation_prompt=True, tokenize=False
    )
    print(f"\n### 2단계 함수 선택/응답 생성을 위한 프롬프트 구성:\n{prompt}")

    first_output = model.generate([prompt], sampling_params_func_call)[0].outputs[0].text
    print(f"\n### 3단계 함수 호출/초기 응답을 위한 LLM 응답:\n{first_output}")

    assistant_msg = parse_tool_calls(first_output)
    messages.append(assistant_msg)
    print(f"\n### 4단계 함수 호출 내용 파싱 및 메시지 추가:\n{assistant_msg}")

    if assistant_msg.get("tool_calls"):
        for call in assistant_msg["tool_calls"]:
            fn = call["function"]["name"]
            args = call["function"]["arguments"]
            if fn == "get_kospi_stock_info":
                result = get_kospi_stock_info(args["stock_name_or_code"])
            else:
                result = json.dumps({"error": "지원하지 않는 함수"}, ensure_ascii=False)
            print(f"\n### 5단계 함수 실행 결과 ({fn}):\n{result}")
            messages.append({
                "role": "tool",
                "tool_call_id": call["id"],
                "name": fn,
                "content": result
            })

        final_prompt = tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=False
        )
        print(f"\n### 6단계 최종 응답 생성을 위한 프롬프트:\n{final_prompt}")

        final_output = model.generate([final_prompt], sampling_params_text)[0].outputs[0].text
        final_response = final_output.strip().rstrip("<|im_end|>")
        print(f"\n### 7단계 최종 LLM 응답 (정리 후):\n{final_response}")
        return final_response
    else:
        print("LLM이 함수를 호출하지 않았습니다. 초기 응답을 반환합니다.")
        content = assistant_msg.get("content", "").strip()
        return content or "응답 내용을 찾을 수 없습니다."

In [9]:
if __name__ == "__main__":
    queries = [
        "SK텔레콤의 주가를 알려줘",
#        "삼성전자 주가 얼마야?"
    ]

    for query in queries:
        print(f"\n========================================")
        print(f" 질문: {query}")
        print(f"==========================================")
        response = query_kospi_info(query)
        print(f"\n 답변: {response}")


 질문: SK텔레콤의 주가를 알려줘

### 1단계 초기 메시지 작성:
[{'role': 'system', 'content': 'You are a helpful assistant. Current Date: 2025-04-21'}, {'role': 'user', 'content': 'SK텔레콤의 주가를 알려줘'}]

### 2단계 함수 선택/응답 생성을 위한 프롬프트 구성:
<|im_start|>system
You are a helpful assistant. Current Date: 2025-04-21

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_kospi_stock_info", "description": "특정 KOSPI 주식의 현재 가격 및 기본 정보를 가져옵니다.", "parameters": {"type": "object", "properties": {"stock_name_or_code": {"type": "string", "description": "주식 이름(예: 'SK텔레콤') 또는 종목 코드(예: '017670')"}}, "required": ["stock_name_or_code"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
SK텔레콤의 주가를

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s, est. speed input: 479.23 toks/s, output: 61.96 toks/s]



### 3단계 함수 호출/초기 응답을 위한 LLM 응답:
<tool_call>
{"name": "get_kospi_stock_info", "arguments": {"stock_name_or_code": "SK텔레콤"}}
</tool_call>

### 4단계 함수 호출 내용 파싱 및 메시지 추가:
{'role': 'assistant', 'tool_calls': [{'type': 'function', 'function': {'name': 'get_kospi_stock_info', 'arguments': {'stock_name_or_code': 'SK텔레콤'}}, 'id': 'call_0'}]}

### 5단계 함수 실행 결과 (get_kospi_stock_info):
{"ticker": "017670.KS", "stock_name": "SKTelecom", "current_price": 57700.0, "previous_close": 57900.0, "currency": "KRW"}

### 6단계 최종 응답 생성을 위한 프롬프트:
<|im_start|>system
You are a helpful assistant. Current Date: 2025-04-21<|im_end|>
<|im_start|>user
SK텔레콤의 주가를 알려줘<|im_end|>
<|im_start|>assistant
<tool_call>
{"name": "get_kospi_stock_info", "arguments": {"stock_name_or_code": "SK텔레콤"}}
</tool_call><|im_end|>
<|im_start|>user
<tool_response>
{"ticker": "017670.KS", "stock_name": "SKTelecom", "current_price": 57700.0, "previous_close": 57900.0, "currency": "KRW"}
</tool_response><|im_end|>
<|im_start|>assistant



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s, est. speed input: 225.75 toks/s, output: 73.17 toks/s]


### 7단계 최종 LLM 응답 (정리 후):
SK텔레콤의 주가는 현재 57,700원입니다. 전일 대비로는 200원 하락했습니다. 화폐 단위는 한국 원(KRW)입니다.

 답변: SK텔레콤의 주가는 현재 57,700원입니다. 전일 대비로는 200원 하락했습니다. 화폐 단위는 한국 원(KRW)입니다.
